In [1]:
# Import Libraries

import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import pandas_ta as ta    # https://github.com/twopirllc/pandas-ta

In [2]:
# User Inputs

assetCodes = ['^RUT']
assetNames = ['Russel2000']

#assetCodes = ['CL=F', 'GC=F', '^RUT', '^GSPC', 'BTC-USD', 'ETH-USD', 'EURUSD=X', 'GBPJPY=X']
#assetNames = ['crudeOil', 'Gold', 'Russel2000', 'S&P500', 'BTC-USD', 'ETH-USD', 'EUR-USD', 'GBP-JPY' ]

start_date = '2006-06-30'
end_date = '2022-11-01'

# Time Interval
timePeriod="max"
weekly = "1wk"
daily="1d"

# Weekly Indicators
weeklyEMAShort = 5
weeklyEMALong = 13
weeklyADX = 6


# Daily Indicators
dailyEMAShort = 12
dailyEMALong = 26
elderRayLength = 12
bbandsLength = 21
macdFast = 12
macdSlow = 26
macdSignal = 9

In [3]:
# Indicator Names

# Number of Assets
count = len(assetCodes)

# Weekly Indicators
weeklyEMAShortIndicatorName = 'EMA_'+str(weeklyEMAShort)
newWeeklyEMAShortIndicatorName = 'weeklyEMA'+str(weeklyEMAShort)

weeklyEMALongIndicatorName = 'EMA_'+str(weeklyEMALong)
newWeeklyEMALongIndicatorName = 'weeklyEMA'+str(weeklyEMALong)

weeklyADXName = 'ADX_'+str(weeklyADX)
newWeeklyADXName = 'weeklyADX'


# Daily Indicators
dailyEMAShortIndicatorName = 'EMA_'+str(dailyEMAShort)
newDailylyEMAShortIndicatorName = 'dailyEMA'+str(dailyEMAShort)

dailyEMALongIndicatorName = 'EMA_'+str(dailyEMALong)
newDailyEMALongIndicatorName = 'dailyEMA'+str(dailyEMALong)

bullPowerIndicatorName = 'BULLP_'+str(elderRayLength)
newBullPowerIndicatorName = 'BullPower'

bearPowerIndicatorName = 'BEARP_'+str(elderRayLength)
newBearPowerIndicatorName = 'BearPower'

bollingerLowerIndicatorName = 'BBL_'+str(bbandsLength)+'_2.0'
newBollingerLowerIndicatorName = 'lowerBB'

bollingerMiddleIndicatorName = 'BBM_'+str(bbandsLength)+'_2.0'
newBollingerMiddleIndicatorName = 'middleBB'

bollingerUpperIndicatorName = 'BBU_'+str(bbandsLength)+'_2.0'
newBollingerUpperIndicatorName = 'upperBB'

bollingerStdIndicatorName = 'BBB_'+str(bbandsLength)+'_2.0'
newBollingerStdIndicatorName = '2stdBB'

macdIndicatorName = "MACD_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDIndicatorName = "MACDline"

macdHistogramIndicatorName = "MACDh_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDHistogramIndicatorName = "MACDHistogram"

macdSignalIndicatorName = "MACDs_"+str(macdFast)+"_"+str(macdSlow)+"_"+str(macdSignal)
newMACDSignalIndicatorName = "MACDSignal"


In [4]:
# Function to calculate weekly Indicators

def weeklyIndicators(assetCode, timePeriod, weekly, start_date, end_date, weeklyEMAShort, weeklyEMALong, weeklyADX,
                     weeklyEMAShortIndicatorName, weeklyEMALongIndicatorName, weeklyADXName,
                     newWeeklyEMAShortIndicatorName,newWeeklyEMALongIndicatorName, newWeeklyADXName):
    
    # Get Weekly Asset Data
    dfWeekly = pd.DataFrame()
    dfWeekly = dfWeekly.ta.ticker(assetCode, period=timePeriod, interval=weekly)
    dfWeekly = dfWeekly[(dfWeekly.index > start_date) & (dfWeekly.index < end_date)]


    # Create Custom Weekly Strategy
    CustomStrategyWeekly = ta.Strategy(
        name="Weekly Indicators",
        description="Weekly EMA and ADX Indicators",
        ta=[
            {"kind": "ema", "length": weeklyEMAShort},
            {"kind": "ema", "length": weeklyEMALong},
            {"kind": "adx", "length": weeklyADX},
        ]
    )
    
    # Run "Custom Weekly Strategy"
    dfWeekly.ta.strategy(CustomStrategyWeekly)
    dfWeekly=dfWeekly.dropna()
    algoDataWeekly = dfWeekly[[weeklyEMAShortIndicatorName, weeklyEMALongIndicatorName, weeklyADXName]]

    algoDataWeekly = algoDataWeekly.rename({weeklyEMAShortIndicatorName: newWeeklyEMAShortIndicatorName,
                                            weeklyEMALongIndicatorName: newWeeklyEMALongIndicatorName,
                                            weeklyADXName: newWeeklyADXName}, axis=1)
    
    
    return algoDataWeekly

In [5]:
# Function to calculate daily Indicators

def dailyIndicators(assetCode, timePeriod, daily, start_date, end_date, dailyEMAShort, dailyEMALong, elderRayLength,
                    bbandsLength, macdFast, macdSlow, macdSignal, dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                    bullPowerIndicatorName, bearPowerIndicatorName, bollingerLowerIndicatorName, 
                    bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName, macdIndicatorName,
                    macdHistogramIndicatorName, macdSignalIndicatorName, newDailylyEMAShortIndicatorName,
                    newDailyEMALongIndicatorName, newBullPowerIndicatorName, newBearPowerIndicatorName, 
                    newBollingerLowerIndicatorName, newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName,
                    newBollingerStdIndicatorName, newMACDIndicatorName, newMACDHistogramIndicatorName,
                    newMACDSignalIndicatorName):
    
    # Get Daily Asset Data

    dfDaily = pd.DataFrame()
    dfDaily = dfDaily.ta.ticker(assetCode, period=timePeriod, interval=daily)
    dfDaily = dfDaily[(dfDaily.index > start_date) & (dfDaily.index < end_date)]

    # Use the pct_change function to generate returns from close prices
    dfDaily["Actual Returns"] = dfDaily["Close"].pct_change()

    # Drop all NaN values from the DataFrame
    dfDaily = dfDaily.dropna()

    # Initialize the new Signal column
    dfDaily['Signal'] = 0.0

    # When Actual Returns are greater than or equal to 0, generate signal to buy asset long
    dfDaily.loc[(dfDaily['Actual Returns'] >= 0), 'Signal'] = 1

    # When Actual Returns are less than 0, generate signal to sell asset short
    dfDaily.loc[(dfDaily['Actual Returns'] < 0), 'Signal'] = -1


    # Create your own Custom Strategy
    CustomStrategyDaily = ta.Strategy(
        name="Daily Indicators",
        description="daily Trading Indicators",
        ta=[
            {"kind": "ema", "length": dailyEMAShort},
            {"kind": "ema", "length": dailyEMALong},
            {"kind": "eri", "length": elderRayLength},
            {"kind": "bbands", "length": bbandsLength},
            {"kind": "macd", "fast": macdFast, "slow": macdSlow, "signal": macdSignal},
        ]
    )


    # Run "Custom Daily Strategy"
    dfDaily.ta.strategy(CustomStrategyDaily)
    dfDaily=dfDaily.dropna()
    algoDataDaily = dfDaily[['Close', 'Actual Returns','Signal',dailyEMAShortIndicatorName, dailyEMALongIndicatorName,
                         bullPowerIndicatorName, bearPowerIndicatorName, bollingerLowerIndicatorName,
                         bollingerMiddleIndicatorName, bollingerUpperIndicatorName, bollingerStdIndicatorName,
                         macdIndicatorName, macdHistogramIndicatorName, macdSignalIndicatorName]]

    algoDataDaily = algoDataDaily.rename({dailyEMAShortIndicatorName: newDailylyEMAShortIndicatorName,
                            dailyEMALongIndicatorName: newDailyEMALongIndicatorName,
                            bullPowerIndicatorName: newBullPowerIndicatorName,
                            bearPowerIndicatorName: newBearPowerIndicatorName,
                            bollingerLowerIndicatorName: newBollingerLowerIndicatorName,
                            bollingerMiddleIndicatorName: newBollingerMiddleIndicatorName,
                            bollingerUpperIndicatorName: newBollingerUpperIndicatorName,
                            bollingerStdIndicatorName: newBollingerStdIndicatorName,
                            macdIndicatorName: newMACDIndicatorName,
                            macdHistogramIndicatorName: newMACDHistogramIndicatorName,
                            macdSignalIndicatorName: newMACDSignalIndicatorName}, axis=1)
    
    
    return algoDataDaily

In [6]:
# Concatent weekly and daily indicators and clean dataset

def combineData(algoDataDaily, algoDataWeekly):
    algoData = pd.concat([algoDataDaily, algoDataWeekly], axis=1)
    #algoData = algoData.ffill(axis=0)
    algoData = algoData.interpolate(method='linear', limit_direction='forward', axis=0)
    algoData = algoData.dropna()
    
    return algoData

# Save file

def saveFile(algoData, assetName):
    fileName = 'algoData_'+assetName+'.csv'
    algoData.to_csv(fileName)

In [7]:
# Get Technical Analysis and Signal Data for all Assets

for i in range(0,count):

    algoDataWeekly = weeklyIndicators(assetCodes[i], timePeriod, weekly, start_date, end_date, weeklyEMAShort, 
                                  weeklyEMALong, weeklyADX, weeklyEMAShortIndicatorName, weeklyEMALongIndicatorName, 
                                  weeklyADXName, newWeeklyEMAShortIndicatorName,newWeeklyEMALongIndicatorName, 
                                  newWeeklyADXName)



    algoDataDaily = dailyIndicators(assetCodes[i], timePeriod, daily, start_date, end_date, dailyEMAShort, dailyEMALong, elderRayLength,
                                bbandsLength, macdFast, macdSlow, macdSignal, dailyEMAShortIndicatorName,
                                dailyEMALongIndicatorName,bullPowerIndicatorName, bearPowerIndicatorName,
                                bollingerLowerIndicatorName, bollingerMiddleIndicatorName, bollingerUpperIndicatorName,
                                bollingerStdIndicatorName, macdIndicatorName,macdHistogramIndicatorName,
                                macdSignalIndicatorName, newDailylyEMAShortIndicatorName,newDailyEMALongIndicatorName,
                                newBullPowerIndicatorName, newBearPowerIndicatorName, newBollingerLowerIndicatorName,
                                newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName,newBollingerStdIndicatorName,
                                newMACDIndicatorName, newMACDHistogramIndicatorName,newMACDSignalIndicatorName)


    algoData = combineData(algoDataDaily, algoDataWeekly)

    saveFile(algoData, assetNames[i])

In [8]:
# Split the data into training and test datasets

# Assign a copy of the features DataFrame called X
X = algoData[[newWeeklyEMAShortIndicatorName, newWeeklyEMALongIndicatorName, newWeeklyADXName, newDailylyEMAShortIndicatorName,
             newDailyEMALongIndicatorName, newBullPowerIndicatorName, newBearPowerIndicatorName, newBollingerLowerIndicatorName,
             newBollingerMiddleIndicatorName, newBollingerUpperIndicatorName, newBollingerStdIndicatorName,
              newMACDIndicatorName, newMACDHistogramIndicatorName, newMACDSignalIndicatorName]].shift().dropna()


# Create the target set selecting the Returns column and assiging it to y
y = algoData['Actual Returns']

# Select the start of the training period
training_begin = X.index.min()

# Select the ending period for the training data with an offset of 145 months
training_end = X.index.min() + DateOffset(months=145)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]


In [9]:
# MACD Strategy Test
#
#def macdTest(algoData):
#    
#    signal = []
#    
#    for ind in algoData.index:
#    
#        if((algoData['MACDline'][ind] > algoData['MACDSignal'][ind]) & (algoData['MACDline'][ind] > algoData['MACDSignal'][ind].shift(1)) & (algoData['weeklyEMA5'][ind] > algoData['weeklyEMA5'][ind].shift(1)) & (algoData['weeklyEMA13'][ind] > algoData['weeklyEMA13'][ind].shift(1)) & (algoData['weeklyADX'][ind] > 25)):
#        
#            signal.append(1)
#    
#        elif ((algoData['MACDline'][ind] < algoData['MACDSignal'][ind]) & (algoData['MACDline'][ind] < algoData['MACDSignal'][ind].shift(1)) & (algoData['weeklyEMA5'] < algoData['weeklyEMA5'][ind].shift(1)) & (algoData['weeklyEMA13'][ind] < algoData['weeklyEMA13'][ind].shift(1)) & (algoData['weeklyADX'][ind] > 25)):
#
#            signal.append(-1)
#    
#        else:
#        
#            signal.append(0)
#        
#    return signal

In [10]:
# MACD Strategy Function

def macdStrategy(algoData):
    
    dates = algoData.index
    
    algoData = algoData.reset_index(drop=True)
    
    signal = [0]
    
    for ind in range(1, algoData.shape[0]):
    
        if((algoData['MACDline'][ind] > algoData['MACDSignal'][ind]) & (algoData['MACDline'][ind-1] > algoData['MACDSignal'][ind-1]) & (algoData['weeklyEMA5'][ind] > algoData['weeklyEMA5'][ind-1]) & (algoData['weeklyEMA13'][ind] > algoData['weeklyEMA13'][ind-1]) & (algoData['weeklyADX'][ind] > 25)):
        
            signal.append(1)
    
        elif ((algoData['MACDline'][ind] < algoData['MACDSignal'][ind]) & (algoData['MACDline'][ind-1] < algoData['MACDSignal'][ind-1]) & (algoData['weeklyEMA5'][ind] < algoData['weeklyEMA5'][ind-1]) & (algoData['weeklyEMA13'][ind] < algoData['weeklyEMA13'][ind-1]) & (algoData['weeklyADX'][ind] > 25)):

            signal.append(-1)
    
        else:
        
            signal.append(0)
    
    algoData.index = dates
        
    return signal

In [11]:
algoData['MACDStrategy'] = macdStrategy(algoData)

In [19]:
algoData['MACDStrategyReturns'] = algoData['Actual Returns'] * algoData['MACDStrategy'].shift()

In [26]:
macdReturns = (1 + algoData['MACDStrategyReturns']).cumprod().sum()

In [29]:
buyHold = (1+algoData['Actual Returns']).cumprod().sum()

In [32]:
macdReturns

60478.505764791

In [31]:
buyHold

6815.134150774334

In [12]:
# Impulse System

In [13]:
# Elder-Ray 

In [14]:
# Bollinger Bands

In [20]:
algoData.to_csv('algoData.csv')